## Expert Knowledge Worker

### A question answering agent that is an expert knowledge worker
### To be used by employees of Insurellm, an Insurance Tech company
### The agent needs to be accurate and the solution should be low cost.

This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\salma\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for langchain and Chroma and plotly
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE   #Helps with visualization of vectors.
import plotly.graph_objects as go

In [3]:
# price is a factor for our company, so we're going to use a low cost model
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Load environment variables in a file called .env
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase
folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

# Please note:

In the next cell, we split the text into chunks.

2 students let me know that the next cell crashed their computer.  
They were able to fix it by changing the chunk_size from 1,000 to 2,000 and the chunk_overlap from 200 to 400.  
This shouldn't be required; but if it happens to you, please make that change!  
(Note that LangChain may give a warning about a chunk being larger than 1,000 - this can be safely ignored).

_With much thanks to Steven W and Nir P for this valuable contribution._

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [7]:
len(chunks)

123

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, employees, contracts, products


## A sidenote on Embeddings, and "Auto-Encoding LLMs"

We will be mapping each chunk of text into a Vector that represents the meaning of the text, known as an embedding.

OpenAI offers a model to do this, which we will use by calling their API with some LangChain code.

This model is an example of an "Auto-Encoding LLM" which generates an output given a complete input.
It's different to all the other LLMs we've discussed today, which are known as "Auto-Regressive LLMs", and generate future tokens based only on past context.

Another example of an Auto-Encoding LLMs is BERT from Google. In addition to embedding, Auto-encoding LLMs are often used for classification.

### Sidenote

In week 8 we will return to RAG and vector embeddings, and we will use an open-source vector encoder so that the data never leaves our computer - that's an important consideration when building enterprise systems and the data needs to remain internal.

In [9]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk

embeddings = OpenAIEmbeddings()

# If you would rather use the free Vector Embeddings from HuggingFace sentence-transformers
# Then replace embeddings = OpenAIEmbeddings()
# with:
# from langchain.embeddings import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [10]:
# Check if a Chroma Datastore already exists - if so, delete the collection to start from scratch
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [ ]:
# Create our Chroma vectorstore!

# Alternative Way 1 :- It helps us create the chroma datastore and populate with Vector embeddings of our Knowledge Base
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)

# Alternative Way 2 :- The below code generates a local Chroma database and chunks into vectors using add_documents.
# vectorstore = Chroma(collection_name='example_collection', embedding_function=embeddings, persist_directory=db_name)
# vectorstore.add_documents(documents=chunks)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 123 documents


In [24]:
print(vectorstore.get()['metadatas'][0])

{'doc_type': 'company', 'source': 'knowledge-base\\company\\about.md'}


In [26]:
# Get one vector and find how many dimensions it has
collection = vectorstore._collection
# With the below code, we convert each chunk to the vector number.
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


## Visualizing the Vector Store

Let's take a minute to look at the documents and their embedding vectors to see what's going on.

In [ ]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
print(result)

# We use the np library for the visualization part.
vectors = np.array(result['embeddings'])

print(vectors)
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

{'ids': ['349b37d9-33ed-41ec-a80a-5ff470345087', '400abdab-becf-4a2c-b616-3e2613f04b77', 'be459ef8-d9f2-4acd-b82f-fffcd7f5e757', 'dbeadfd5-dd2b-4972-985d-3298593d015e', '58c9e722-4bbd-4f56-a041-0de9d61f4338', '7071d644-57a5-47eb-8919-4e31f6f6156a', '4dcd0aea-eab4-46a8-a85b-7053d893cae3', '99025d3f-5066-46f0-b2e4-abee1fa7ddb9', '2543f7e0-7f8c-4ad1-a133-0d27b43b58b7', 'ddbc7e74-8cab-42f1-979f-45e347d707d8', '6f980dc5-5fbf-40cd-9a3d-35494bff65cc', '1075483e-dc29-4c4f-af4e-6026a0ee718b', '48800a05-daf4-4ef0-b708-76ace7ee70de', '2a14ad7e-d5e7-4588-9694-7236e285d09f', '46c71de9-2052-41e6-a6b4-70322d3be203', 'd8b7d09c-0e85-4b5a-a3d9-ed8b4893e41a', 'e132d207-4ec6-45a8-914d-95526cdb8b21', '6b6e6715-2318-48bb-a426-01377f2305af', '40e1f1df-03e4-4316-87bc-816d3a5b6e17', 'cb06304f-36da-49f3-bdca-827a63fccbb0', 'b9a5c59a-c5f6-4fad-89ab-5d2fcb4c3f8e', '9d3a901a-24b8-477f-bd91-58d9f2195012', '1bcf45b7-194b-41ad-9a59-f3ad426ef5b8', 'bc112541-5749-4ace-8ab0-5e118f3dfb51', 'e0265a75-24b7-429f-98d7-6742c7

In [ ]:
# We humans find it easier to visalize things in 2D!
# Reduce the dimensionality of the vectors to 2D using t-SNE
# (t-distributed stochastic neighbor embedding)

# This helps in understanding the vector embeddings in a 2D scatter plot & the closest points to each other define 
# the high possibility of having similar meaning with each other. 
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    # Showing the first 100 characters of each document in the hover text.
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

# fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [orange, orange, orange, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue],
                         'opacity': 0.8,
                         'size': 5},
              'mode': 'markers',
              'text': [Type: company<br>Text: # About Insurellm  Insurellm was
                       founded by Avery Lancaster in 2015 as an insurance tech
                       startup des..., Type: company<br>Text: # Careers at
                       Insurellm  Insurellm is hiring! We are looking for talented
                       software engineers, data sc..., Type: company<br>Text: #
                       Overview of Insurellm  Insurellm is an innovative insurance
                       tech firm with 200 employees across th..., Type:
                       contracts<br>Text: # Contract with Apex Reinsurance for
                       Rellm: AI-Powered Enterprise Reinsurance Solution  ## Terms
                       1...., Type: contracts<br>Text: ## Renewal  1. **Automatic
                       Renewal**: This Agreement will automatically renew for
                       successive one-yea..., Type: contracts<br>Text: 2.
                       **Seamless Integrations**: The architecture of Rellm allows
                       for easy integration with existing sy..., Type:
                       contracts<br>Text: 1. **Technical Support**: Provider shall
                       offer dedicated technical support to the Client via phone,
                       ..., Type: contracts<br>Text: **Insurellm, Inc.**
                       _____________________________   Authorized Signature
                       Date: ________________..., Type: contracts<br>Text: #
                       Contract with Belvedere Insurance for Markellm  ## Terms
                       This Contract ("Agreement") is made and e..., Type:
                       contracts<br>Text: ## Renewal 1. **Renewal Terms**: This
                       Agreement may be renewed for additional one-year terms upon
                       mu..., Type: contracts<br>Text: ## Features 1. **AI-Powered
                       Matching**: Belvedere Insurance will benefit from Markellm's
                       AI-powered ..., Type: contracts<br>Text: ## Support 1.
                       **Technical Support**: Technical support will be available
                       from 9 AM to 7 PM EST, Mond..., Type: contracts<br>Text:
                       **Belvedere Insurance**   Signature: ______________________
                       Name: [Authorized Signatory]   Title: ..., Type:
                       contracts<br>Text: # Contract with BrightWay Solutions for
                       Markellm  **Contract Date:** October 5, 2023   **Contract
          

In [39]:
# Let's try 3D!
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=20, b=10, l=10, t=40)
)

# fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': [orange, orange, orange, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, red, red, red, red, red, red,
                                   red, red, red, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, green, green, green, green, green, green,
                                   green, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue, blue, blue,
                                   blue, blue, blue, blue, blue, blue],
                         'opacity': 0.8,
                         'size': 5},
              'mode': 'markers',
              'text': [Type: company<br>Text: # About Insurellm  Insurellm was
                       founded by Avery Lancaster in 2015 as an insurance tech
                       startup des..., Type: company<br>Text: # Careers at
                       Insurellm  Insurellm is hiring! We are looking for talented
                       software engineers, data sc..., Type: company<br>Text: #
                       Overview of Insurellm  Insurellm is an innovative insurance
                       tech firm with 200 employees across th..., Type:
                       contracts<br>Text: # Contract with Apex Reinsurance for
                       Rellm: AI-Powered Enterprise Reinsurance Solution  ## Terms
                       1...., Type: contracts<br>Text: ## Renewal  1. **Automatic
                       Renewal**: This Agreement will automatically renew for
                       successive one-yea..., Type: contracts<br>Text: 2.
                       **Seamless Integrations**: The architecture of Rellm allows
                       for easy integration with existing sy..., Type:
                       contracts<br>Text: 1. **Technical Support**: Provider shall
                       offer dedicated technical support to the Client via phone,
                       ..., Type: contracts<br>Text: **Insurellm, Inc.**
                       _____________________________   Authorized Signature
                       Date: ________________..., Type: contracts<br>Text: #
                       Contract with Belvedere Insurance for Markellm  ## Terms
                       This Contract ("Agreement") is made and e..., Type:
                       contracts<br>Text: ## Renewal 1. **Renewal Terms**: This
                       Agreement may be renewed for additional one-year terms upon
                       mu..., Type: contracts<br>Text: ## Features 1. **AI-Powered
                       Matching**: Belvedere Insurance will benefit from Markellm's
                       AI-powered ..., Type: contracts<br>Text: ## Support 1.
                       **Technical Support**: Technical support will be available
                       from 9 AM to 7 PM EST, Mond..., Type: contracts<br>Text:
                       **Belvedere Insurance**   Signature: ______________________
                       Name: [Authorized Signatory]   Title: ..., Type:
                       contracts<br>Text: # Contract with BrightWay Solutions for
                       Markellm  **Contract Date:** October 5, 2023   **Contract
          